Understand Problem & RF Classifier Solution

**Goal/Task**: Utilize Python SciKit-learn's "random forest" machine learning "classification" algorithm to build a model that will predict if a business opportunity in Salesforce will be Closed Won or Closed Lost in advance of pursuing the opportunity, so that salespeople can prioritize opportunities with greater win probability. 

**Random forest** consists of a large number of individual (and uncorrelated) decision trees that operate as an "ensemble"; each decision tree makes a prediction, and class with most votes becomes model's prediction. 

**Define Problem Type**: Is this problem a classification or regression problem? A. Binary Classification problem- we will predict the class "closed won" or "closed lost" with probability assigned.




In [ ]:
# Load Libraries, Packages & Modules
import pandas as pd # provides data processing; Pandas dataframe to organize data, pandas to import data from csv
import pickle # serializes your machine learning algorithms and save the serialized format to a file
import io #enables writing to the unicode data; will need this to handle csv file
import numpy as np #creates multidimensional arrays, linear algebra
import seaborn as sns #makes graphs/visuals
# from allows me to import certain functions from a module (in contrast, numpy by default imports all packages in a simple import, but sklearn is too big to follow the this approach)
from sklearn.model_selection import train_test_split #splits dataset into training and testing sets
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV  #automates hyperparamater tuning
from sklearn.ensemble import RandomForestClassifier #imports random forest classifier model; Ensemble method is to combine many individual models (i.e decision trees) to achieve better prediction accuracy
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder #use this for unordered data to convert strings to numbers in the data, otherwise sklearn cannot handle
from sklearn.compose import ColumnTransformer #allows you to selectively apply data preparation transforms (i.e. specify which columns of data to onehotencode)
from sklearn.pipeline import make_pipeline, Pipeline #use to chain steps together; takes data, preprocesses it, builds model
# from sklearn.metrics import metrics #metrics module for evaluation of model/accuracy calculation; metrics would not import. 
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

In [ ]:
# Optional Code. Version Control
print ("Different versions of python, sklearn, pandas, etc. have different capabilities") 
print ("---------")
import sys
print ("Python Intepreter and (GCC) Compiler Version:") 
print(sys.version)
print ("---------")
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))
print ("---------")
print ("What Version of Pandas:") 
pd.show_versions()

Create the data source 
*   Tips on how to select or exclude features? a. features that are redundant or highly correlated are candidates for exclusion b. using domain specific knowledge, it should be logical that the independant features selected could predict closed won/lost
*   Random Forest has its own built-in feature selection tools, and is naturally resistant to non-informative predictors
*   More high-quality data will make this model more accurate
*   For this project, I used Tableau which is connected to my data source, pulled in relevant fields, and exported data as xlsx.
*   Data Filters used: Total Dollar Amt of each Opp at least 1 dollar, 10x10=T, Stage=Closed Won or Closed Lost only




In [ ]:
# Load Data
# tip: colab support multiple import options incl. local file, mount google drive, sql, aws s3, etc. 
from google.colab import drive #this method mounts google drive 
drive.mount('/content/drive')

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/foldername/writeyourfilenamehere.xlsx') # read in dataset stored in google drive and store it as pandas' dataframe
# tip: click folder icon in colab's left pane->click mount drive; also can navigate to file and click copy path to easily pull file path 

In [ ]:
# Understand Your Data & Check Data for Missing Values
# tip: df.columns->shows column names; df.head() prints first 5 lines; df.drop() removes a data column; df.shape -> (#rows, #columns); df.isnull().sum()->shows # of null values for columns, df.dtypes->shows what data types we have
print ("First 5 Rows of Data, All Columns:")
df.head()
print ("---------")
print ("Data Types by Column:") #int=integer, float=floating point number, boolean=true/false, object=string Or mixed string-number
df.dtypes
print ("---------")
print ("Dataset's (# Rows, # Columns):")
df.shape
print ("---------")
print ("# of Null Values by Column:")
df.isnull().sum()
print ("---------")
print ("# of Null Values by Column:") #I have no nulls since I fixed the data in advance of this project, however, if I did I could use df.dropna().shape to drop drows or use an alternative imputation method
df.isnull().sum()
print ("---------")
print ("Percentage of Nulls by Column:")
df.isnull().sum()*100/ len(df)
print ("---------")
print ("List of All Columns:")
df.columns


In [ ]:
# Divide data into Dependent and Independent Variables
# y=dependent variable aka "labels", and X=set of independent variables aka "features" or "attributes"
y = df['Stage'] # alternative way to write: y = df.Stage OR y = df.iloc[:,0].values
X = df[['Opportunity Type']] # note: see step 11 to understand why I ended up using only 1 independant variable.
# alternative way to write: X = df.drop('Stage', axis='columns') OR X = df.iloc[:,1:].values

In [ ]:
# Optional Code. Are y and X correctly assigned? 
print ('Are y and X variables correctly assigned?')
print ('What the dataframe returns for y:')
print (y)
print ("---------")
print ("The rows, columns for y:")
print (y.shape)
print ("---------")
print ("What the dataframe returns for X:")
print (X)
print ("---------")
print ("The rows, columns for X:")
print (X.shape)

In [ ]:
# Split data into training and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33 , random_state=3, stratify=y, shuffle=True) # test set size is 33% and remaining training set size is 67%

# test size: depends on data size and # of classes/features, but as a rule of thumb: use 70/30 Train/Test split if data<30k samples, use 90/10 Train/Test split if data>1M samples. 
# random_state: fixes the random number generator of the model; you could set it to any non-negative integer
# stratify=y: makes sure that you keep the same proportion of y output values through the training and test sets; stratify is for classification problems when you may not have a balanced number of examples for each class label
# shuffle=False: turns off data shuffling and random split so that results are reproducible each time code is run


Build Data Pre-processing Pipeline, Define & Fit Model
*   [scikit-learn.org/0.21/auto_examples/compose/plot_column_transformer_mixed_types.html](https://) and [kaggle.com/alexisbcook/pipelines](https://) and https://youtu.be/irHhDMbw3xo



In [ ]:
numeric_features = [] # tip: int, floating, boolean are often numeric; use df.dtypes to get datatypes by column and look at data itself df.head() 
categorical_features = ['Opportunity Type']
dep_var = ['Stage'] # dependent variable treated separately 

In [ ]:
# Preprocessing for numerical data
numeric_transformer = SimpleImputer(strategy='median') #impute missing values with median for numerical columns
# tip: scaling Not necessary for random forest, and other decision tree models.

In [ ]:
# Preprocessing for categorical data
categorical_transformer = OneHotEncoder(handle_unknown='ignore') #One Hot Encode the categorical columns to convert text values to numbers, otherwise ML aglo cannot process.


In [ ]:
# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer( #columntranformer allows us to selectively apply data transform
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])


In [ ]:
# Define rf model as the preprocess pipeline + append rf classifier
model = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier())]) 

In [ ]:
# Fit Model on Training Datasets
# https://stackoverflow.com/questions/62699521/should-you-fit-train-test-or-all-x-and-y-values-for-a-linearregression, youtube.com/watch?v=s7vsF2bzQX8, datacamp.com/community/tutorials/random-forests-classifier-python
model.fit(X_train, y_train) #use fit method to fit/train model on training data only; test data needs to remain separate so it is not influenced by training data. 

In [ ]:
# Evaluate Accuracy of Model's Predictions
# medium.com/analytics-vidhya/accuracy-vs-f1-score, scikit-learn.org/stable/modules/model_evaluation.html, machinelearningmastery.com/what-is-imbalanced-classification/, programcreek.com/python/example/81623/sklearn.metrics.classification_report
# Does my dataset have an imbalanced class distribution? my y variable splits into approx. 35% won and 65% lost which would classify as a slight imbalance and can often be treated like a normal classification problem. A severe imbalance is more like severe imbalance 1:100 or more. 
predictions = model.predict(X_test)
print ("What is the Accuracy of this Model?") #this gives me what model accuracy is before tuning.
print ("Precision = TruePositives / (TruePositives + FalsePositives). Best value at 1 and worst score at 0; Precision helps minimize false positives")
print ("Recall = TruePositives / (TruePositives + FalseNegatives). Best value at 1 and worst score at 0; recall helps minimize false negatives")
print ("F1 Score = weighted average of both precision and recall into a single measure. Best value at 1 and worst score at 0")
print ("---------")
print (classification_report(y_test, predictions))
print ("Accuracy on training set:") # accuracy is measure of all correctly identified cases; con- if problem is highly imbalanced, you get high accuracy score simply by predicting all observations belong to majority class.
print (model.score(X_train, y_train)) #% training accuracy of the model 
print ("---------")
print ("Accuracy on testing set:")
print (model.score(X_test, y_test)) #% testing accuracy of the model (0.89=89%)
print ("---------")
#print ("Confusion Matrix: shows accuracy of model as applied to test data & generates # of True Negatives and False Positives in row1 and False Negatives and True Positives in row2")
#print ("True Negative-Model Correctly Predicts Negative Class, False Positive- Model incorrectly predicts positive class")
#print ("False Negative=incorrectly predicts negative class, True Positive- Correctly Predicts Positive Class")
#print (confusion_matrix(y_test, y_pred)) #shows true positive, true negative, false positive, false negative

Simplify Model. Strip out independant variables one at a time, rerun model, and test accuracy. Originally, I had a very complex model with numerous independant variables. However, I kept stripping them out and rerunning/refitting this model, and then testing the accuracy above. As it turned out, my model achieved nearly the same accuracy with 1 independant variable as it did with the original 10+ independant variables. I opted for the simplified model based on 1 independant variable for numerous reasons: easier to explain and gain adoption of model with varying business lines, model runs faster, & critically result of simplified model provided better business intelligence on the most important factors resulting in the particular classification. Simple>Complex.

In [ ]:
# Intepret Results
df[['Stage', 'Opportunity Type']].groupby(['Stage', 'Opportunity Type']).size() # returns the number of samples of my independant var across the 2 categories of the dependant var; easy from here to generate %s and analyze results